In [1]:
import pandas as pd
import re

In [2]:
# import two datasets with confirmed bandwidth

rpi_datetime = '2023-09-20_09-06-07'
laptop_datetime = '2023-09-20_11-43-32'

df_rpi = pd.read_csv(f'dfs/{rpi_datetime}.csv')
df_laptop = pd.read_csv(f'dfs/{laptop_datetime}.csv')

In [3]:
with open(f'log/debug_{rpi_datetime}.log', 'r') as file:
        rpi_bandwith = float(re.search(r"Bandwidth:\s+(\d+\.\d+)\s+Mbits/sec", file.readlines()[27]).group(1))
print(f'{rpi_bandwith=} Mbits/sec')

with open(f'log/debug_{laptop_datetime}.log', 'r') as file:
        laptop_bandwith = float(re.search(r"Bandwidth:\s+(\d+\.\d+)\s+Mbits/sec", file.readlines()[27]).group(1))
print(f'{laptop_bandwith=} Mbits/sec')

rpi_bandwith=56.81 Mbits/sec
laptop_bandwith=17.79 Mbits/sec


In [4]:
# get the total bits sent and received
# btw, client.total.sent == server.total.received and server.total.sent == client.total.received

# ABY benchmarks are in bytes -> x8 to get bits -> div by million to get Mbits
rpi_mbits_sent = (df_rpi['server.communication.total.sent'].mean() * 8) / 1e6
rpi_mbits_received = (df_rpi['server.communication.total.received'].mean() * 8) / 1e6
print(f"{rpi_mbits_sent=} Mbits, {rpi_mbits_received=} Mbits")

laptop_mbits_sent = (df_laptop['server.communication.total.sent'].mean() * 8) / 1e6
laptop_mbits_received = (df_laptop['server.communication.total.received'].mean() * 8) / 1e6
print(f"{laptop_mbits_sent=} Mbits, {laptop_mbits_received=} Mbits")

rpi_mbits_sent=222.09306738461538 Mbits, rpi_mbits_received=222.088944 Mbits
laptop_mbits_sent=222.09305728 Mbits, laptop_mbits_received=222.088944 Mbits


In [5]:
# compute the time that it takes to send over data
# communication time = total size sent and received / bandwidth

rpi_comm_time = (rpi_mbits_received + rpi_mbits_sent) / rpi_bandwith
print(f'{rpi_comm_time=} sec')

laptop_comm_time = (laptop_mbits_received + laptop_mbits_sent) / laptop_bandwith
print(f'{laptop_comm_time=} sec')

rpi_comm_time=7.818729297388055 sec
laptop_comm_time=24.968072022484545 sec


In [6]:
# print ABY timings benchmarks for rpi (in seconds)
timings_col = [col for col in df_rpi if col.startswith('server.timings')]
df_rpi[timings_col].mean() / 1000

server.timings.total          41.463946
server.timings.init            0.000643
server.timings.circuitgen      0.000548
server.timings.network         0.031972
server.timings.baseots         1.678600
server.timings.setup           9.155909
server.timings.otextension     9.113954
server.timings.garbling        0.000012
server.timings.online         32.308025
dtype: float64

In [7]:
# print ABY timings benchmarks for laptop
df_laptop[timings_col].mean() / 1000

server.timings.total          62.840146
server.timings.init            0.000078
server.timings.circuitgen      0.000100
server.timings.network         0.217183
server.timings.baseots         0.153283
server.timings.setup          19.943892
server.timings.otextension    19.942091
server.timings.garbling        0.000001
server.timings.online         42.896251
dtype: float64